In [1]:
import os.path as osp
import time

import torch
import torch.nn.functional as F
from sklearn.metrics import f1_score

from torch_geometric.datasets import PPI
from torch_geometric.loader import DataLoader
from torch_geometric.nn import SAGEConv  # Importing SAGEConv layer

path = './ppi_data/'
train_dataset = PPI(path, split='train')
val_dataset = PPI(path, split='val')
test_dataset = PPI(path, split='test')
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)


class GraphSAGEPPI(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = SAGEConv(train_dataset.num_features, 256)
        self.conv2 = SAGEConv(256, 256)
        self.conv3 = SAGEConv(256, train_dataset.num_classes)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = self.conv3(x, edge_index)
        return x


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
loss_op = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)


def train():
    model.train()

    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        loss = loss_op(model(data.x, data.edge_index), data.y)
        total_loss += loss.item() * data.num_graphs
        loss.backward()
        optimizer.step()
    return total_loss / len(train_loader.dataset)


@torch.no_grad()
def test(loader):
    model.eval()

    ys, preds = [], []
    for data in loader:
        ys.append(data.y)
        out = model(data.x.to(device), data.edge_index.to(device))
        preds.append((out > 0).float().cpu())

    y, pred = torch.cat(ys, dim=0).numpy(), torch.cat(preds, dim=0).numpy()
    return f1_score(y, pred, average='micro') if pred.sum() > 0 else 0


times = []
for epoch in range(1, 101):
    start = time.time()
    loss = train()
    val_f1 = test(val_loader)
    test_f1 = test(test_loader)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_f1:.4f}, '
          f'Test: {test_f1:.4f}')
    times.append(time.time() - start)
print(f"Median time per epoch: {torch.tensor(times).median():.4f}s")


Epoch: 001, Loss: 0.5802, Val: 0.3808, Test: 0.3809
Epoch: 002, Loss: 0.5458, Val: 0.4447, Test: 0.4464
Epoch: 003, Loss: 0.5224, Val: 0.4931, Test: 0.4995
Epoch: 004, Loss: 0.4951, Val: 0.5665, Test: 0.5748
Epoch: 005, Loss: 0.4691, Val: 0.5637, Test: 0.5764
Epoch: 006, Loss: 0.4456, Val: 0.6160, Test: 0.6293
Epoch: 007, Loss: 0.4278, Val: 0.5882, Test: 0.6047
Epoch: 008, Loss: 0.4124, Val: 0.6303, Test: 0.6466
Epoch: 009, Loss: 0.3940, Val: 0.6364, Test: 0.6558
Epoch: 010, Loss: 0.3800, Val: 0.6592, Test: 0.6790
Epoch: 011, Loss: 0.3680, Val: 0.6713, Test: 0.6913
Epoch: 012, Loss: 0.3615, Val: 0.6959, Test: 0.7154
Epoch: 013, Loss: 0.3471, Val: 0.6878, Test: 0.7085
Epoch: 014, Loss: 0.3387, Val: 0.6788, Test: 0.7002
Epoch: 015, Loss: 0.3284, Val: 0.7066, Test: 0.7291
Epoch: 016, Loss: 0.3239, Val: 0.7280, Test: 0.7487
Epoch: 017, Loss: 0.3251, Val: 0.7025, Test: 0.7264
Epoch: 018, Loss: 0.3129, Val: 0.7160, Test: 0.7405
Epoch: 019, Loss: 0.3019, Val: 0.7412, Test: 0.7644
Epoch: 020, 